In [141]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

In [7]:
# webscraping helper functions
def prep_driver(user_agent: str):
    "prepare webdriver options, returns object of class 'selenium.webdriver.chrome.options.Options'"
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument("--incognito")
    options.add_argument("--start-maximized")
    options.add_argument("--enable-automation")
    # options.add_argument("--window-size=1420,1080")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-gpu")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable')
    options.add_argument(f"user-agent={user_agent}")
    return options

def get_user_agent()->str:
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value,OperatingSystem.MAC.value,OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems,limit=100)
    user_agent = user_agent_rotator.get_random_user_agent()
    return user_agent

In [8]:
# set user agent to imitate browser requests
user_agent: str = get_user_agent()

get all products & urls for products

In [115]:
# initialize df for products & urls
init_df = pd.DataFrame(columns = ['product', 'url'])
product_list = []
url_list = []

# general code for getting all product elements on page (this is page 1)
options = prep_driver(user_agent)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
url = "https://www.traderjoes.com/home/products/category/cheese-29"
driver.get(url)
path = '//main/div/div/div/div[1]/div/div/div[1]/div[2]/section/ul/li'
products = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, path)))
page = driver.page_source
soup = BeautifulSoup(page, 'html')
product_els = soup.find_all('li', class_="ProductList_productList__item__1EIvq")

# extracting first page product list
for el in product_els:
	product_list.append(el.find_all('a', href=True)[2].get_text())
	url_list.append(el.find_all('a', href=True)[2]['href'])



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\linds\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
C:\Users\linds\AppData\Local\Temp\ipykernel_7724\3066782374.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)


In [117]:
# find next page buttons: `buttons`
pages_path = "//main/div/div/div/div[1]/div/div/div[1]/div[2]/section/div[2]/ul"
next_page_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, pages_path)))
driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, pages_path)))
buttons_path = "//main/div/div/div/div[1]/div/div/div[1]/div[2]/section/div[2]/ul/li"
buttons = driver.find_elements(By.XPATH, buttons_path)

In [118]:
# clicking through pages 2 -> end = len(buttons) - 1 <- doesn't include right arrow button
# & extracting all data from each page
for i in range(1,len(buttons)-1):
	new_button_path = buttons_path + "[" + str(i+1) + "]"
	new_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, new_button_path)))
	driver.execute_script("arguments[0].scrollIntoView();", new_button)
	WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, new_button_path)))
	driver.execute_script("arguments[0].click();", new_button)
	path = '//main/div/div/div/div[1]/div/div/div[1]/div[2]/section/ul/li'
	products = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, path)))
	page = driver.page_source
	soup = BeautifulSoup(page, 'html')
	product_els = soup.find_all('li', class_="ProductList_productList__item__1EIvq")
	for el in product_els:
		product_list.append(el.find_all('a', href=True)[2].get_text())
		url_list.append(el.find_all('a', href=True)[2]['href'])

In [125]:
# populate initialized dataframe with products
init_df = pd.DataFrame({'product' : product_list, 'url' : url_list})
num_rows = len(init_df)


'/home/products/pdp/all-natural-fresh-mozzarella-cheese-077726'

In [250]:
# get all poss nutrition categories & save cat in `all_nutr_cat`
# weird categories happened at row index 2,19,20,26

wait = WebDriverWait(driver, 10)
tj_url = "https://www.traderjoes.com"
all_nutr_cat = []
nutrition_path = "//main/div/div/div[2]/div[1]/div/div[3]/div[1]/div[1]/div[2]/div/table/tbody/tr"
for index, row in init_df.iterrows():
	curr_url = tj_url + row['url']
	driver.get(curr_url)
	try:
		nutrition = wait.until(EC.presence_of_element_located((By.XPATH, nutrition_path)))
		nutrition = driver.find_elements(By.XPATH, nutrition_path)
		for i in range(len(nutrition)):
			el = nutrition[i]
			children = el.find_elements(By.XPATH, "*")
			nutr_stat = []
			nutr_cat = children[0].text
			if nutr_cat not in all_nutr_cat:
				all_nutr_cat.append(nutr_cat)
				# print('index:', index) # print index of where new cat occurs (for debugging)
			# for j in range(1, len(children)):
			# 	nutr_stat.append(children[j].text)
	except Exception as e:
		print('exception:', e)

index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 0
index: 2
index: 19
index: 20
exception: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00799943+2595139]
	Ordinal0 [0x0072C9F1+2148849]
	Ordinal0 [0x00624528+1066280]
	Ordinal0 [0x00650FD4+1249236]
	Ordinal0 [0x006511CB+1249739]
	Ordinal0 [0x0067D812+1431570]
	Ordinal0 [0x0066BA34+1358388]
	Ordinal0 [0x0067BAF2+1424114]
	Ordinal0 [0x0066B806+1357830]
	Ordinal0 [0x00646086+1204358]
	Ordinal0 [0x00646F96+1208214]
	GetHandleVerifier [0x0093B232+1658114]
	GetHandleVerifier [0x009F312C+2411516]
	GetHandleVerifier [0x0082F261+560433]
	GetHandleVerifier [0x0082E366+556598]
	Ordinal0 [0x0073286B+2173035]
	Ordinal0 [0x007375F8+2192888]
	Ordinal0 [0x007376E5+2193125]
	Ordinal0 [0x007411FC+2232828]
	BaseThreadInitThunk [0x759B6739+25]
	RtlGetFullPathName_UEx [0x76EE8E7F+1215]
	RtlGetFullPathName_UEx [0x76EE8E4D+1165]

index: 26


In [262]:
# creating new lists for each category
nutr_var = [x.lower() for x in all_nutr_cat]
nutr_var = [x.replace(" ", "_") for x in nutr_var]

# print str code to create var lists
for x in nutr_var:
	print(x + " = [None] * num_rows")

# print str code to create var dict mapping
for i in range(len(all_nutr_cat)):
	print("\'" + all_nutr_cat[i] + "\' : " + nutr_var[i] + ",")

'Total Fat' : total_fat,
'Saturated Fat' : saturated_fat,
'Trans Fat' : trans_fat,
'Cholesterol' : cholesterol,
'Sodium' : sodium,
'Total Carbohydrate' : total_carbohydrate,
'Dietary Fiber' : dietary_fiber,
'Total Sugars' : total_sugars,
'Includes' : includes,
'Protein' : protein,
'Vitamin D' : vitamin_d,
'Calcium' : calcium,
'Iron' : iron,
'Potassium' : potassium,
'Fiber' : fiber,
'Vitamin A' : vitamin_a,
'Added Sugars' : added_sugars,
'Sugars' : sugars,


In [263]:
# create a dictionary mapping str to list
total_fat = [None] * num_rows
saturated_fat = [None] * num_rows
trans_fat = [None] * num_rows
cholesterol = [None] * num_rows
sodium = [None] * num_rows
total_carbohydrate = [None] * num_rows
dietary_fiber = [None] * num_rows
total_sugars = [None] * num_rows
includes = [None] * num_rows
protein = [None] * num_rows
vitamin_d = [None] * num_rows
calcium = [None] * num_rows
iron = [None] * num_rows
potassium = [None] * num_rows
fiber = [None] * num_rows
vitamin_a = [None] * num_rows
added_sugars = [None] * num_rows
sugars = [None] * num_rows

nutr_var_dict = {'Total Fat' : total_fat,
'Saturated Fat' : saturated_fat,
'Trans Fat' : trans_fat,
'Cholesterol' : cholesterol,
'Sodium' : sodium,
'Total Carbohydrate' : total_carbohydrate,
'Dietary Fiber' : dietary_fiber,
'Total Sugars' : total_sugars,
'Includes' : includes,
'Protein' : protein,
'Vitamin D' : vitamin_d,
'Calcium' : calcium,
'Iron' : iron,
'Potassium' : potassium,
'Fiber' : fiber,
'Vitamin A' : vitamin_a,
'Added Sugars' : added_sugars,
'Sugars' : sugars}

In [270]:
# get all nutrition info data for each row/cheese in `init_df`
# appends `None` if data cannot be found or exception is thrown

wait = WebDriverWait(driver, 10)
tj_url = "https://www.traderjoes.com"
price_path = "//main/div/div/div[1]/div[1]/div/div[2]/div[1]/div/div/span[1]"
price_weight_path = "//main/div/div/div[1]/div[1]/div/div[2]/div[1]/div/div/span[2]"
serving_path = "//main/div/div/div[2]/div[1]/div/div[3]/div[1]/div[1]/div[2]/div/div/div[1]/div[2]/sub"
cal_path = "//main/div/div/div[2]/div[1]/div/div[3]/div[1]/div[1]/div[2]/div/div/div[2]/div[2]"
# nutrition
nutrition_path = "//main/div/div/div[2]/div[1]/div/div[3]/div[1]/div[1]/div[2]/div/table/tbody/tr"

# initial data: price, weight, serving size, calories
price = []
price_weight = []
serving_size = [] # in grams
cal = []
for index, row in init_df.iterrows():
	curr_url = tj_url + row['url']
	driver.get(curr_url)
	try:
		curr_price = wait.until(EC.presence_of_element_located((By.XPATH, price_path)))
		curr_price = driver.find_element(By.XPATH, price_path)
		price.append(curr_price.text)
	except:
		price.append(None)

	try:
		curr_price_weight = wait.until(EC.presence_of_element_located((By.XPATH, price_weight_path)))
		curr_price_weight = driver.find_element(By.XPATH, price_weight_path)
		price_weight.append(curr_price_weight.text)
	except:
		price_weight.append(None)

	try:
		curr_serving_size = wait.until(EC.presence_of_element_located((By.XPATH, serving_path)))
		curr_serving_size = driver.find_element(By.XPATH, serving_path)
		serving_size.append(curr_serving_size.text)
	except:
		serving_size.append(None)

	try:
		curr_cal = driver.find_element(By.XPATH, cal_path)
		cal.append(curr_cal.text)
	except:
		cal.append(None)

	try:
		nutrition = wait.until(EC.presence_of_element_located((By.XPATH, nutrition_path)))
		nutrition = driver.find_elements(By.XPATH, nutrition_path) # all nutrition cat elements
		for i in range(len(nutrition)):
			nutr_stat = []
			el = nutrition[i] # curr nutrition el
			children = el.find_elements(By.XPATH, "*")
			nutr_cat = children[0].text # curr nutrition el cat name
			for j in range(1, len(children)): # its children are grams & % daily val (stat)
				nutr_stat.append(children[j].text) # append stat
			nutr_var_dict.get(nutr_cat)[index] = nutr_stat # put all stat for curr el in dict
	except Exception as e:
		print("Exception:", e, "happened at row", index)

Exception: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00799943+2595139]
	Ordinal0 [0x0072C9F1+2148849]
	Ordinal0 [0x00624528+1066280]
	Ordinal0 [0x00650FD4+1249236]
	Ordinal0 [0x006511CB+1249739]
	Ordinal0 [0x0067D812+1431570]
	Ordinal0 [0x0066BA34+1358388]
	Ordinal0 [0x0067BAF2+1424114]
	Ordinal0 [0x0066B806+1357830]
	Ordinal0 [0x00646086+1204358]
	Ordinal0 [0x00646F96+1208214]
	GetHandleVerifier [0x0093B232+1658114]
	GetHandleVerifier [0x009F312C+2411516]
	GetHandleVerifier [0x0082F261+560433]
	GetHandleVerifier [0x0082E366+556598]
	Ordinal0 [0x0073286B+2173035]
	Ordinal0 [0x007375F8+2192888]
	Ordinal0 [0x007376E5+2193125]
	Ordinal0 [0x007411FC+2232828]
	BaseThreadInitThunk [0x759B6739+25]
	RtlGetFullPathName_UEx [0x76EE8E7F+1215]
	RtlGetFullPathName_UEx [0x76EE8E4D+1165]
 happened at row 23


In [274]:
add_df1 = pd.DataFrame({'price' : price, 'per weight' : price_weight,
						'serving size' : serving_size, 'calories' : cal})
add_df2 = pd.DataFrame(nutr_var_dict)

,product,url
0,All Natural Fresh Mozzarella Cheese,/home/products/pdp/all-natural-fresh-mozzarell...
1,Fancy Shredded Mexican Style Cheese Blend,/home/products/pdp/fancy-shredded-mexican-styl...
2,Spicy Buffalo Cheddar,/home/products/pdp/spicy-buffalo-cheddar-073840
3,Quattro Formaggi,/home/products/pdp/quattro-formaggi-052954
4,Garlic Bread Cheese,/home/products/pdp/garlic-bread-cheese-071622


In [299]:
# save uncleaned data into csv
final_df = pd.concat([init_df, add_df1, add_df2], axis=1, ignore_index=False)
final_df.to_csv('uncleaned_df.csv', index=False)